In [1]:
# importing the required modules
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import time
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999
df = pd.read_csv('../Data/profile_links.csv')
df["0"]
pl = list(df["0"])



In [121]:
basic_data = []
main_data = []
product_review = []
profile_all = []
all_project = []
review_all = []
main_category = []



for i in range(20):
    one_user = []
    print("Serial :",i)
    url = pl[i]
    print(url)
    driver = webdriver.Chrome(r'C:\Users\USER\chromedriver_win32\chromedriver.exe')
    driver.get(url)
    time.sleep(5)
    
    ######################################### basic data section ########################################
    #basic informations
    basic = []
    source_code = driver.page_source
    soup = BeautifulSoup(source_code, 'html.parser')
    
    #name
    name = soup.find_all('strong', class_ = 'userName--1ZA07')
    for n in name:
        basic.append(n.text)   
    
    #category
    category = soup.find_all('strong', class_ = 'introCategory--F81Ky')
    for e in category:
        basic.append(e.text)
        
    #specialty   
    ba = []
    sp = soup.find_all('div', class_ = 'categoryName--1zWtA')
    for m in sp:
        ba.append(m.text)
    basic.append(ba)
    
    #rating
    rating = soup.find_all('div', class_ = 'itemRating--360UA itemRating--2-rFv typeLarge--1cEMN')
    for k in rating:
        km = k.text
        basic.append(km.replace("평균 평점", ""))
        
        #Reviews and consultations
        reviews = soup.find_all('span', class_ = 'statsNum--32OX2')
        for kk in reviews:
            basic.append(kk.text)
    
    #appending basic data of all user
    basic_data.append(basic)
    
    
    ######################################### main ########################################
    ### main info data for one user
    maininfo = []
    uh = ["대표자","상호명","사업자등록번호","통신판매업번호", "사업장 주소", "고객센터",'메일']

    #main section info
    nn = []
    infos = soup.find_all('ul', class_ = 'productInfoList--1-H-D')
    for f in infos:
        li = f.find_all('li')
        #each list item
        for i in range(len(li)):
            ii = li[i]
            val = uh[i]
            head = ii.find_all("span", class_ = "title--2YCH3")
            maini = ii.find_all("span", class_ = "text--1z2Eb")
            for h in head:
                if h.text != val:
                    if [k, " "] not in nn:
                        nn.append("NA")
                else:
                    for j in maini:
                        if j.text not in nn:
                            if j.text == None or j.text == "" or j.text == " ":
                                nn.append("NA")
                            else:   
                                nn.append(j.text)
    main_data.append(nn)
    
    
    ######################################### count product section ########################################
    #count product and review section
    products = []
    tt = soup.find_all('div', class_ = "list--e6w5E")
    for t in tt:  
        cc = t.find_all('a', class_='item--3Oz2i')
        for cd in cc:
            ce = cd.find_all('div', class_ = "count--2w5o6")
            for i in ce:
                products.append(i.text)
    product_review.append(products)
    
    
    ######################################### Profile data section ########################################
    #profile informations
    profile_heading = []
    profile_text = []
    firm_name = []
    firm_text = []

    div = soup.find_all('div', class_ = 'sectionIntroduce--3_qQB')
    for heading in div: 
        indiv = heading.find_all('div', class_ = 'introduceMain--g3aND')
        for i in indiv:
            head = i.find_all('strong', class_ = 'introduceMainTitle--2MZc-')
            for h in head:
                profile_heading.append(h.text)
                
            text = i.find_all('p', class_ = 'introduceText--2R5pY')
            for ii in text:
                profile_text.append(ii.text)
                
    careerdiv = soup.find_all('div', class_ = ['profileCareer--3_uFh','isExpert--2GkDA'])
    for i in careerdiv:
        cd = i.find_all('div', class_ = 'profileBox--1jlog')
        for j in cd:
            cd = j.find_all('div', class_ = 'careerJob--2-hX4')
            for c in cd:
                firm_name.append(c.text)
            
            cui = j.find_all('ul', class_ = 'careerList--2dpZg')
            for cc in cui:
                firm_text.append(cc.text)
                
    profile_all.append([profile_heading, profile_text, firm_name, firm_text])      
        


#     ######################################### review section ########################################
    #review object
    review_obj = []
    reviews_user = []
    reviews_rating = []
    reviews_heading = []
    reviews_text = []
    
    rdiv = soup.find_all('div', class_ = "listSection--kViCl")
    for eachr in rdiv:
        ee = eachr.find_all('div', class_ = "reviewItem--1OwNO")
        
        for each in ee:
            name = each.find_all('span', class_ = ["item--3sQA9 ","nickname--2OOe6"])
            for nm in name:
                reviews_user.append(nm.text)
                
            rating = each.find_all('div', class_ = ["expertPoint--2Zrvr","expertPoint--13H3V"])
            for r in rating:
                reviews_rating.append(r.text)

            head = each.find_all('div', class_ = "reviewTitle--qv3Pk")
            for r in head:
                reviews_heading.append(r.text)

            commentdiv = each.find_all('p', class_ = "reviewText--28mzN")
            for ecom in commentdiv:
                reviews_text.append(ecom.text)
                    
    for i in range(3):
        try:
            review_obj.append(reviews_user[i])
            if "평점" in reviews_rating[i]:
                rating = reviews_rating[i].replace("평점", "")
                review_obj.append(rating)
            else:
                review_obj.append(reviews_rating)
            review_obj.append(reviews_heading[i])
            review_obj.append(reviews_text[i])
        except:
            continue
            
    review_all.append(review_obj)

    ######################################### driver close section ########################################

    driver.quit()
    ######################################### Final dataframe section ########################################

#basic dataframe section
basicdf = pd.DataFrame(basic_data)
basicdf.columns = ["Name","subcategory","Specialty","Rating","Reviews","Consultations"]

#main dataframe section
maindf = pd.DataFrame(main_data)
maindf.columns =["Representative", "Company_name", "Business_registration_number", "Mail_order_number", "Business_address", "Customer_Center",'Mail']

#product review dataframe section
prdf = pd.DataFrame(product_review)
prdf.columns =["Products", "Reviews"]

# # profile dataframe section
profiledf = pd.DataFrame(profile_all)
profiledf.columns =["Profile", "Details", "Firm", "Education/Career"]

reviewdf = pd.DataFrame(review_all)
reviewdf.columns =["Customer_name_1", "Customer_rating_1", "Review_heading_1", "Review_text_1", "Reviewer_name_2", "Reviewer_rating_2", "Review_heading_2", "Review_text_2","Reviewer_name_3", "Reviewer_rating_3", "Review_heading_3", "Review_text_3"]
                    
                    
for i in range(len(basicdf["subcategory"])):
    if basicdf["subcategory"][i] in ['법률','노동/노무','지식재산/특허',"등기/공탁/법무",'민원/행정']:
        main_category.append(["법률", basicdf["subcategory"][i]])

    elif row[3] in ['세금/세무','회계/감사','통관/관세','온라인 마케팅','온라인쇼핑몰','엑스퍼트 사업','경영/기술컨설팅','유통관리','가맹점창업','건축','번역/통역','날씨컨설팅','원가 분석']:
        main_category.append(["비즈니스", basicdf["subcategory"][i]])
        
    elif row[3] in ['자산컨설팅','부동산 상담','손해사정','신용상담','감정평가']:
        main_category.append(["금융/재테크", basicdf["subcategory"][i]])
        
    elif row[3] in ['심리상담','영양/다이어트','MBTI ']:
        main_category.append(["건강", basicdf["subcategory"][i]])
        
    elif row[3] in ['운세/사주','타로카드','작명','꿈해몽','관상','풍수']:
        main_category.append(["운세", basicdf["subcategory"][i]])
        
    elif row[3] in ['펫 관리','연애','육아','명상','패션/스타일','뷰티','요리/홈쿠킹','커피/주류','인테리어','청소/세탁','교통사고 분석','자동차수리']:
        main_category.append(["생활", basicdf["subcategory"][i]])
        
    elif row[3] in ['음악/악기','미술/디자인','공예/공방','무용/ 발레','사진','실용/방송댄스','뮤지컬/공연','낚시','원예/홈가드닝','여행','글쓰기/논술']:
        main_category.append(["취미", basicdf["subcategory"][i]])
        
    elif row[3] in ['외국어학습','입시/진학','해외유학','대학교학습','고등학교학습','중학교학습','초등학교학습']:
        main_category.append(["교육/학습", basicdf["subcategory"][i]])
        
    elif row[3] in ['피트니스','골프','필라테스','요가','생활스포츠','자전거','수상 스포츠','동계 스포츠','유아체육']:
        main_category.append(["운동/스포츠", basicdf["subcategory"][i]])
        
    elif row[3] in ['게임하우투','IT노하우','코딩','오피스문서','동영상 제작']:
        main_category.append(["IT/컨텐츠", basicdf["subcategory"][i]])
        
    elif row[3] in ['라이프 코칭','취업','자기PR','공무원시험 ','자격증시험']:
        main_category.append(["자기계발", basicdf["subcategory"][i]])
        
    else:
        main_category.append(["네이버고객센터", basicdf["subcategory"][i]])
categorydf = pd.DataFrame(main_category)
categorydf.columns = ["Main_category", "Sub_category"]

#merging the dataframes
frames = [categorydf, basicdf, maindf, prdf, profiledf, reviewdf]
df = pd.concat(frames, axis = 1)

Serial : 0
https://m.kin.naver.com/profile/index.nhn?u=WK98lR8DRpn%2FLLywxgaB0212FN0EJLGFtlWJJWbR4o4%3D
Serial : 1
https://m.kin.naver.com/profile/index.nhn?u=zvRvZSrTBCeYUJjRbEc5OKhzAkyWWga1TYilww4ty7Y%3D
Serial : 2
https://m.kin.naver.com/profile/index.nhn?u=WkbFz5ahgf%2BXv5lZQmIlFCGBxzU1D27QZrWw6mJbLQE%3D
Serial : 3
https://m.kin.naver.com/profile/index.nhn?u=nAVNYg%2FFjPFN%2BpRS%2Fm3pDV3tbZPaThbGt%2BiDng1DS%2Bk%3D
Serial : 4
https://m.kin.naver.com/profile/index.nhn?u=SbT2p9Ppk9cNV%2FZVxO77eQmi4l6DvTV0la%2FRK3scqco%3D
Serial : 5
https://m.kin.naver.com/profile/index.nhn?u=RjMOip8ykso5IMKvoXSV26u4w3J6UU0XpN93iH%2Fit24%3D
Serial : 6
https://m.kin.naver.com/profile/index.nhn?u=178cUY3BLVoMV1G3V62JO0y0ZINNKvqWNGQzEBkHELk%3D
Serial : 7
https://m.kin.naver.com/profile/index.nhn?u=3Bvy0VqcKnrNRX4YO%2Fa%2BP7ttjfLnqnkC%2FDuEsDIT4S0%3D
Serial : 8
https://m.kin.naver.com/profile/index.nhn?u=HpCgShzsisofg7hsSczP3Hv8XqeYjQo2fZ4ezsPahU8%3D
Serial : 9
https://m.kin.naver.com/profile/index.nhn?u=o

In [126]:
df

,0,1,Name,subcategory,Specialty,Rating,Reviews,Consultations,Representative,Company_name,Business_registration_number,Mail_order_number,Business_address,Customer_Center,Mail,Products,Reviews,Profile,Details,Firm,Education/Career,Customer_name_1,Customer_rating_1,Review_heading_1,Review_text_1,Reviewer_name_2,Reviewer_rating_2,Review_heading_2,Review_text_2,Reviewer_name_3,Reviewer_rating_3,Review_heading_3,Review_text_3
0,법률,법률,이승준 변호사,법률,"[부동산, 건축, 가족, 이혼, 민사사건]",4.9,854,"1,035",김주덕,법무법인 태일,214-86-62026,-,"(우:06595) 서울특별시 서초구 법원로3길 25 (서초동, 태흥빌딩) 2-5층",02-595-4142,leesjlo@naver.com,8,187,"[30년경력/ 세무사자격/ 대형로펌,사시출신,친절명확상담]","[-전국경제인연합회 고문변호사-도곡렉슬 재건축 조합,황학구역,돈의문구역 재개발조합 ...",[법무법인 태일구성원 변호사],"[전국경제인연합회 고문변호사도곡렉슬아파트 재건축 조합, 황학구역,돈의문구역,한남구역...",choi****,5,[부동산/재건축재개발/임대차상담 및 관련세무법률]전화상담,정말 만족스러운 상담이었습니다~!! 명확하고 친절하게 설명해셔서 감사드려용,qas0****,5,[부동산/재건축재개발/임대차상담 및 관련세무법률]전화상담,명확한 설명 감사합니다,neone,5,양도 증여 상속세등 세무법률전화상담,대만족합니다 감사합니다!
1,법률,법률,이재용,법률,"[손해배상, 교통사고, 형사사건]",4.9,735,961,이재용,JY법률사무소,128-32-34769,-,"(우:06647) 서울특별시 서초구 서초대로 254 (서초동, 오퓨런스) 14층",025824833,wjdgml1278@naver.com,3,735,[사시출신 경력14년차 의뢰인과 함께하는 형사전문변호사],[JY법률사무소 대표 이재용변호사입니다. 저는 14년 동안 의뢰인들이 처한 법률문제...,[JY법률사무소 대표 변호사형사사건 변호사],[2020 한국소비자평가 KCA 우수 전문인\rJY 법률사무소\r변호사 이재용 법률...,Vbmun,5,신속하고 정확한 형사사건 법률상담,덕분에 마음이 놓였습니다 항상 친절하게 답변해주셔서 감사합니다~,Vbmun,5,명쾌하고 꼼꼼한 20분 심층 전화상담,평점높은후기가 많길래 왤까 궁금햇는데 이유를 알게 됐습니다 다른건 몰라도 전문성 하...,baab****,5,성범죄 상담,피해자 입장에서 친절하고 명확하게 조언을 해주셔서 신뢰가 가고 의사결정에 도움이 되...
2,법률,법률,이용희,법률,"[손해배상, 형사사건, 민사사건]",4.9,2084,"3,242",이용희,변호사 이용희 법률사무소,585-70-00218,-,"(우:06605) 서울특별시 서초구 서초중앙로 160 (서초동, 법률센터) 505호",02-575-8859,macoshark@naver.com,4,"2,084",[서울대/사시/대한변협인증 형•민사전문],"[▶▶보내주시는 신뢰, 실력으로 보답하겠습니다.◀◀]",[법률사무소 천지 - 로시컴 협력 변호사 변호사],[現 법률사무소 천지 대표변호사\r現 (주)덕광제이앤에이치 자문\r現 (주)세운5구...,None,None,None,None,None,None,None,None,None,None,None,None
3,법률,법률,유지원,법률,"[가족, 이혼, 형사사건, 민사사건]",4.9,299,593,NA,NA,NA,NA,None,None,None,5,299,"[풍부한 경험, 다수성공사례 바탕 변호사 법률상담]",[ ],[법무법인 와이케이 변호사변호사],[대한변호사협회 가사/형사 분야 전문등록\r현) 서울지방법원 국선변호인\r현) 서울...,하이얍,5,가사전문변호사 가사 상담,"정보를 알아봐주시면서까지 너무 열심히, 좋은 답변해주셔서 감사드려요. 정말 감사드린...",NJ755 com,5,10분 법률 상담 [채팅/음성],"너무 좋았습니다~!!!!!! 답변도 빠르시고, 명확하게 설명 해주셔서 많은 도움이 ...",made****,5,가사전문변호사 가사 상담,친절하고 신속히 답변주셔서 문제해결에 큰 도움이 되었습니다
4,법률,법률,박재성,법률,"[부동산, 건축, 형사사건, 민사사건]",5.0,40,96,NA,NA,NA,NA,None,None,None,2,40,[박재성 변호사 (TEL 010 4826 7096)],[Office Tel : 010-4826-7096 / Fax : 02-525-800...,[법무법인 정향 소속변호사변호사],"[인텔리콘 법률사무소\r법무법인 김앤김\r법무법인 피너클\r, 고려대학교 전기전자전...",flig****,5,법률심층상담(30분이상),일주일동안 너무나 힘든 시기에서 상담 받고 나서는 마음이 편해지고 위로가 되었습니다...,파랑스머프,5,법률심층상담,대출은 함부로 하는기 아니라는걸 알았어요. 명확하고 핵심적인 내용 설명 이 좋았습니...,dldb****,5,법률심층상담(30분이상),너무 친절하시고 상담하는 내내 잘 해주셨습니다^^
5,법률,법률,지세훈 변호사,법률,"[가족, 이혼, 손해배상, 형사사건]",5.0,66,147,지세훈,변호사지세훈법률사무소,601-49-65260,2020-서울송파-3003호,(우:05548) 서울특별시 송파구 오금로 97 (방이동) 6층,01029350324,anlawyerg@gmail.com,10,59,[[쾌속대응] 풍부한 경험에서 나오는 노련한 법률상담],[명예훼손 / 모욕 / 고소대리이혼 / 재산분할 / 위자료양육비 증액 / 감액 사건...,[법률사무소 중현파트너변호사],[법률사무소 중현 파트너변호사법무부 법무심의관실 공익법무관대한법률구조공단 수원지부 ...,bh17****,5,"[모욕, 명예훼손] 확실하고 단호한 법률상담",친절하시고 설명도 현실적으로 잘해주셔서 좋았습니다,ekwj****,5,"[모욕, 명예훼손] 확실하고 단호한 법률상담",궁금했던걸 확실히 알려주셔서 감사합니다^^,dlgk****,5,"[모욕, 명예훼손] 확실하고 단호한 법률상담",친절하시고 설명도 잘해주심니다
6,법률,법률,곽소현 변호사,법률,"[부동산, 건축, 손해배상, 민사사건]",4.9,164,242,곽소현,수평선 법률사무소,269-03-01201,-,"(우:06644) 서울특별시 서초구 반포대로28길 94 (서초동, Gwell HOM...",02-522-6323,522-6323@naver.com,4,157,[[민사법 전문변호사] 부동산 관련 주요 상담 진행],"[민사법 전문 변호사로서, 부동산 계약에 관한 법률 상담부터 소송, 경매, 등기까지...",[수평선 법률사무소변호사],[],pobi****,5,"[전화상담] 부동산 관련 계약 해제, 손해배상 민사 상담",월세 계약 해지 관련해서 변호사님께 전화 상담을 받았어요. 저의 상황에 맞는 핵심 ...,coco****,5,"[전화상담] 부동산 관련 계약 해제, 손해배상 민사 상담",상세하고 친절한 설명 감사합니다^^!,apac****,5,"[전화상담] 부동산 관련 계약 해제, 손해배상 민사 상담",친절하고 명쾌한 답변 감사합니다.법을 쉽게 해석해주고 알아 듣기 편하게 답변 주셔서...
7,법률,법률,권오훈 대표변호사,법률,"[부동산, 건축, 가족, 이혼, 형사사건]",5.0,2162,"3,308",권오훈,법률사무소 훈,101-11-94132,-,"(우:03157) 서울특별시 종로구 종로 19 (종로1가, 르메이에르종로타운1) 9...",027337112,cleansea5h@hoonoffice.com,7,"1,424","[서울법대, 사법시험 출신의 경험많은 변호사]","[언제나 고객의 입장에서 최선의 해법을 제시해 드립니다권오훈 변호사는 서울법대, 사...",[법률사무소 훈대표변호사],"[법률사무소 훈 대표변호사서울특별시 공익변호사노동법

In [127]:
df.to_csv("Data.csv")

In [128]:
df = pd.read_csv('Data.csv')
df = df.drop(['Unnamed: 0'], axis = 1) 
df

,0,1,Name,subcategory,Specialty,Rating,Reviews,Consultations,Representative,Company_name,Business_registration_number,Mail_order_number,Business_address,Customer_Center,Mail,Products,Reviews.1,Profile,Details,Firm,Education/Career,Customer_name_1,Customer_rating_1,Review_heading_1,Review_text_1,Reviewer_name_2,Reviewer_rating_2,Review_heading_2,Review_text_2,Reviewer_name_3,Reviewer_rating_3,Review_heading_3,Review_text_3
0,법률,법률,이승준 변호사,법률,"['부동산, 건축', '가족, 이혼', '민사사건']",4.9,854,"1,035",김주덕,법무법인 태일,214-86-62026,-,"(우:06595) 서울특별시 서초구 법원로3길 25 (서초동, 태흥빌딩) 2-5층",02-595-4142,leesjlo@naver.com,8,187,"['30년경력/ 세무사자격/ 대형로펌,사시출신,친절명확상담']","['-전국경제인연합회 고문변호사-도곡렉슬 재건축 조합,황학구역,돈의문구역 재개발조합...",['법무법인 태일구성원 변호사'],"['전국경제인연합회 고문변호사도곡렉슬아파트 재건축 조합, 황학구역,돈의문구역,한남구...",choi****,5.0,[부동산/재건축재개발/임대차상담 및 관련세무법률]전화상담,정말 만족스러운 상담이었습니다~!! 명확하고 친절하게 설명해셔서 감사드려용,qas0****,5.0,[부동산/재건축재개발/임대차상담 및 관련세무법률]전화상담,명확한 설명 감사합니다,neone,5.0,양도 증여 상속세등 세무법률전화상담,대만족합니다 감사합니다!
1,법률,법률,이재용,법률,"['손해배상', '교통사고', '형사사건']",4.9,735,961,이재용,JY법률사무소,128-32-34769,-,"(우:06647) 서울특별시 서초구 서초대로 254 (서초동, 오퓨런스) 14층",025824833,wjdgml1278@naver.com,3,735,['사시출신 경력14년차 의뢰인과 함께하는 형사전문변호사'],['JY법률사무소 대표 이재용변호사입니다. 저는 14년 동안 의뢰인들이 처한 법률문...,['JY법률사무소 대표 변호사형사사건 변호사'],['2020 한국소비자평가 KCA 우수 전문인\rJY 법률사무소\r변호사 이재용 법...,Vbmun,5.0,신속하고 정확한 형사사건 법률상담,덕분에 마음이 놓였습니다 항상 친절하게 답변해주셔서 감사합니다~,Vbmun,5.0,명쾌하고 꼼꼼한 20분 심층 전화상담,평점높은후기가 많길래 왤까 궁금햇는데 이유를 알게 됐습니다 다른건 몰라도 전문성 하...,baab****,5.0,성범죄 상담,피해자 입장에서 친절하고 명확하게 조언을 해주셔서 신뢰가 가고 의사결정에 도움이 되...
2,법률,법률,이용희,법률,"['손해배상', '형사사건', '민사사건']",4.9,2084,"3,242",이용희,변호사 이용희 법률사무소,585-70-00218,-,"(우:06605) 서울특별시 서초구 서초중앙로 160 (서초동, 법률센터) 505호",02-575-8859,macoshark@naver.com,4,"2,084",['서울대/사시/대한변협인증 형•민사전문'],"['▶▶보내주시는 신뢰, 실력으로 보답하겠습니다.◀◀']",['법률사무소 천지 - 로시컴 협력 변호사 변호사'],['現 법률사무소 천지 대표변호사\r現 (주)덕광제이앤에이치 자문\r現 (주)세운5...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,법률,법률,유지원,법률,"['가족, 이혼', '형사사건', '민사사건']",4.9,299,593,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,299,"['풍부한 경험, 다수성공사례 바탕 변호사 법률상담']",[' '],['법무법인 와이케이 변호사변호사'],['대한변호사협회 가사/형사 분야 전문등록\r현) 서울지방법원 국선변호인\r현) 서...,하이얍,5.0,가사전문변호사 가사 상담,"정보를 알아봐주시면서까지 너무 열심히, 좋은 답변해주셔서 감사드려요. 정말 감사드린...",NJ755 com,5.0,10분 법률 상담 [채팅/음성],"너무 좋았습니다~!!!!!! 답변도 빠르시고, 명확하게 설명 해주셔서 많은 도움이 ...",made****,5.0,가사전문변호사 가사 상담,친절하고 신속히 답변주셔서 문제해결에 큰 도움이 되었습니다
4,법률,법률,박재성,법률,"['부동산, 건축', '형사사건', '민사사건']",5.0,40,96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,40,['박재성 변호사 (TEL 010 4826 7096)'],['Office Tel : 010-4826-7096 / Fax : 02-525-80...,['법무법인 정향 소속변호사변호사'],"['인텔리콘 법률사무소\r법무법인 김앤김\r법무법인 피너클\r', '고려대학교 전기...",flig****,5.0,법률심층상담(30분이상),일주일동안 너무나 힘든 시기에서 상담 받고 나서는 마음이 편해지고 위로가 되었습니다...,파랑스머프,5.0,법률심층상담,대출은 함부로 하는기 아니라는걸 알았어요. 명확하고 핵심적인 내용 설명 이 좋았습니...,dldb****,5.0,법률심층상담(30분이상),너무 친절하시고 상담하는 내내 잘 해주셨습니다^^
5,법률,법률,지세훈 변호사,법률,"['가족, 이혼', '손해배상', '형사사건']",5.0,66,147,지세훈,변호사지세훈법률사무소,601-49-65260,2020-서울송파-3003호,(우:05548) 서울특별시 송파구 오금로 97 (방이동) 6층,01029350324,anlawyerg@gmail.com,10,59,['[쾌속대응] 풍부한 경험에서 나오는 노련한 법률상담'],['명예훼손 / 모욕 / 고소대리이혼 / 재산분할 / 위자료양육비 증액 / 감액 사...,['법률사무소 중현파트너변호사'],['법률사무소 중현 파트너변호사법무부 법무심의관실 공익법무관대한법률구조공단 수원지부...,bh17****,5.0,"[모욕, 명예훼손] 확실하고 단호한 법률상담",친절하시고 설명도 현실적으로 잘해주셔서 좋았습니다,ekwj****,5.0,"[모욕, 명예훼손] 확실하고 단호한 법률상담",궁금했던걸 확실히 알려주셔서 감사합니다^^,dlgk****,5.0,"[모욕, 명예훼손] 확실하고 단호한 법률상담",친절하시고 설명도 잘해주심니다
6,법률,법률,곽소현 변호사,법률,"['부동산, 건축', '손해배상', '민사사건']",4.9,164,242,곽소현,수평선 법률사무소,269-03-01201,-,"(우:06644) 서울특별시 서초구 반포대로28길 94 (서초동, Gwell HOM...",02-522-6323,522-6323@naver.com,4,157,['[민사법 전문변호사] 부동산 관련 주요 상담 진행'],"['민사법 전문 변호사로서, 부동산 계약에 관한 법률 상담부터 소송, 경매, 등기까...",['수평선 법률사무소변호사'],[],pobi****,5.0,"[전화상담] 부동산 관련 계약 해제, 손해배상 민사 상담",월세 계약 해지 관련해서 변호사님께 전화 상담을 받았어요. 저의 상황에 맞는 핵심 ...,coco****,5.0,"[전화상담] 부동산 관련 계약 해제, 손해배상 민사 상담",상세하고 친절한 설명 감사합니다^^!,apac****,5.0,"[전화상담] 부동산 관련 계약 해제, 손해배상 민사 상담",친절하고 명쾌한 답변 감사합니다.법을 쉽게 해석해주고 알아 듣기 편하게 답변 주셔서...
7,법률,법률,권오훈 대표변호사,법률,"['부동산, 건축', '가족, 이혼', '형사사건']",5.0,2162,"3,308",권오훈,법률사무소 훈,101-11-94132,-,"(우:03157) 서울특별시 종로구 종로 19 (종로1가, 르메이에르종로타운1) 9...",027337112,cleansea5h@hoonoffice.com,7,"1,424","['서울법대, 사법시험 출신

In [109]:
basicdf

,Name,subcategory,Specialty,Rating,Reviews,Consultations
0,이승준 변호사,법률,"[부동산, 건축, 가족, 이혼, 민사사건]",4.9,854,"1,035"
1,이재용,법률,"[손해배상, 교통사고, 형사사건]",4.9,734,961
2,이용희,법률,"[손해배상, 형사사건, 민사사건]",4.9,2084,"3,241"
3,유지원,법률,"[가족, 이혼, 형사사건, 민사사건]",4.9,299,593
4,박재성,법률,"[부동산, 건축, 형사사건, 민사사건]",5.0,40,96


In [114]:
basicdf["subcategory"][1]

'법률'

In [116]:
categorydf

,0,1
0,법률,법률
1,법률,법률
2,법률,법률
3,법률,법률
4,법률,법률


In [91]:
maindf

,Representative,Company_name,Business_registration_number,Mail_order_number,Business_address,Customer_Center,Mail
0,김주덕,법무법인 태일,214-86-62026,-,"(우:06595) 서울특별시 서초구 법원로3길 25 (서초동, 태흥빌딩) 2-5층",02-595-4142,leesjlo@naver.com
1,이재용,JY법률사무소,128-32-34769,-,"(우:06647) 서울특별시 서초구 서초대로 254 (서초동, 오퓨런스) 14층",025824833,wjdgml1278@naver.com
2,이용희,변호사 이용희 법률사무소,585-70-00218,-,"(우:06605) 서울특별시 서초구 서초중앙로 160 (서초동, 법률센터) 505호",02-575-8859,macoshark@naver.com
3,NA,NA,NA,NA,None,None,None
4,NA,NA,NA,NA,None,None,None


In [ ]:
prdf

In [46]:
reviewdf

,Name,subcategory,Specialty,Rating,Reviews,Consultations
0,이승준 변호사,법률,"[부동산, 건축, 가족, 이혼, 민사사건]",4.9,854,"1,035"
1,이재용,법률,"[손해배상, 교통사고, 형사사건]",4.9,734,960
2,이용희,법률,"[손해배상, 형사사건, 민사사건]",4.9,2082,"3,239"
3,유지원,법률,"[가족, 이혼, 형사사건, 민사사건]",4.9,299,593
4,박재성,법률,"[부동산, 건축, 형사사건, 민사사건]",5.0,40,95


In [47]:
frame = [basicdf, maindf]
df = pd.concat(frame, axis = 1)
df

,Name,subcategory,Specialty,Rating,Reviews,Consultations,Representative,Company_name,Business_registration_number,Mail_order_number,Business_address,Customer_Center,Mail
0,이승준 변호사,법률,"[부동산, 건축, 가족, 이혼, 민사사건]",4.9,854,"1,035",김주덕,법무법인 태일,214-86-62026,-,"(우:06595) 서울특별시 서초구 법원로3길 25 (서초동, 태흥빌딩) 2-5층",02-595-4142,leesjlo@naver.com
1,이재용,법률,"[손해배상, 교통사고, 형사사건]",4.9,734,960,이재용,JY법률사무소,128-32-34769,-,"(우:06647) 서울특별시 서초구 서초대로 254 (서초동, 오퓨런스) 14층",025824833,wjdgml1278@naver.com
2,이용희,법률,"[손해배상, 형사사건, 민사사건]",4.9,2082,"3,239",이용희,변호사 이용희 법률사무소,585-70-00218,-,"(우:06605) 서울특별시 서초구 서초중앙로 160 (서초동, 법률센터) 505호",02-575-8859,macoshark@naver.com
3,유지원,법률,"[가족, 이혼, 형사사건, 민사사건]",4.9,299,593,NA,NA,NA,NA,None,None,None
4,박재성,법률,"[부동산, 건축, 형사사건, 민사사건]",5.0,40,95,NA,NA,NA,NA,None,None,None


In [34]:
maindf

,Representative,Company_name,Business_registration_number,Mail_order_number,Business_address,Customer_Center,Mail
0,"[대표자, 이재용]","[상호명, JY법률사무소]","[사업자등록번호, 128-32-34769]","[통신판매업번호, -]","[사업장 주소, (우:06647) 서울특별시 서초구 서초대로 254 (서초동, 오퓨...","[고객센터, 025824833]","[메일, wjdgml1278@naver.com]"
1,"[대표자, 이재용]","[상호명, JY법률사무소]","[사업자등록번호, 128-32-34769]","[통신판매업번호, -]","[사업장 주소, (우:06647) 서울특별시 서초구 서초대로 254 (서초동, 오퓨...","[고객센터, 025824833]","[메일, wjdgml1278@naver.com]"
2,"[대표자, 이용희]","[상호명, 변호사 이용희 법률사무소]","[사업자등록번호, 585-70-00218]","[통신판매업번호, -]","[사업장 주소, (우:06605) 서울특별시 서초구 서초중앙로 160 (서초동, 법...","[고객센터, 02-575-8859]","[메일, macoshark@naver.com]"
3,"[[[평균 평점], [<path d=""M6.5 9.75l-3.82 2.009.73-...",None,None,None,None,None,None
4,"[[[평균 평점], [<path d=""M6.5 9.75l-3.82 2.009.73-...",None,None,None,None,None,None


In [35]:
basicdf

,Name,subcategory,Specialty,Rating,Reviews,Consultations
0,[],None,None,None,None,None
1,이재용,법률,"[손해배상, 교통사고, 형사사건]",4.9,734,960
2,이용희,법률,"[손해배상, 형사사건, 민사사건]",4.9,2082,"3,238"
3,유지원,법률,"[가족, 이혼, 형사사건, 민사사건]",4.9,299,593
4,박재성,법률,"[부동산, 건축, 형사사건, 민사사건]",5.0,40,95


In [ ]:
                          
#         for ll in li:
#             uh = ["대표자","상호명","사업자등록번호","통신판매업번호", "사업장 주소", "고객센터",'메일']
#             for u in range(len(uh)):
#                 if uh[u] in ll.text:
#                     b = uh[u]
#                     la = ll.text
#                     if la == None or la == "":
#                         nn.append(" ")
#                     else:
#                         nn.append(la.replace(b , ""))
    all_user.append(nn)
    
    
#     
            
#     ### Project data for one user
#     projects = soup.find_all('div', class_ = 'listArea--peDdh')
# #   projects  and consultations 
#     all_project = []
#     for y in projects:
#         one = []
#         yy = y.find_all('div', class_ = 'item--1ZJSx')
#         for t in yy:
#             project_item = []
#             tdiv = t.find_all('div', class_ =['itemTitle--2vWBq','elip2--nFWXY'])
#             for td in tdiv:
#                 project_title = td.text
#                 project_item.append(project_title)

#             ratdiv = t.find_all('div', class_ =['itemGroup--2RnIL','ItemGroup_itemGroup--1f-on'])
#             for rd in ratdiv:
#                 rating = rd.text
#                 project_item.append(rating)

#             feediv = t.find_all('span', class_ =['priceInner--1HE2v'])
#             for fd in feediv:
#                 fee = fd.text
#                 project_item.append(fee)
#             one.append([*project_item])   
#         all_project.append([*one])
#     p = []
#     for i in range(len(all_project)):
#         data = all_project[i]
#         for j in range(len(data)):
#             dj = data[j]
#             for k in range(len(dj)):
#                 bb = dj[k]
#                 p.append(bb)


#     #review object
#     rdiv = soup.find_all('div', class_ = "listSection--kViCl")
#     reviews_rating = []
#     reviews_heading = []
#     reviews_text = []

#     for eachr in rdiv:
#         ee = eachr.find_all('div', class_ = "reviewItem--1OwNO")
#         for each in ee:
#             rating = each.find_all('div', class_ = ["expertPoint--2Zrvr","expertPoint--13H3V"])
#             for r in rating:
#                 reviews_rating.append(r.text)

#             head = each.find_all('div', class_ = "reviewTitle--qv3Pk")
#             for r in head:
#                 reviews_heading.append(r.text)

#             commentdiv = each.find_all('p', class_ = "reviewText--28mzN")
#             for ecom in commentdiv:
#                 reviews_text.append(ecom.text)

#     review_obj = []
#     for i in range(len(reviews_heading)):
#         review_obj.append(reviews_heading[i])
#         review_obj.append(reviews_rating[i])
#         review_obj.append(reviews_text[i])

    #final works  
#     main.append(maininfo)
#     all_user.append([url,*basic, *maininfo, *products, *p, *review_obj ])    


In [94]:
all_user

[['김주덕',
  '법무법인 태일',
  '214-86-62026',
  '-',
  '(우:06595) 서울특별시 서초구 법원로3길 25 (서초동, 태흥빌딩) 2-5층',
  '02-595-4142',
  'leesjlo@naver.com'],
 ['김주덕',
  '법무법인 태일',
  '214-86-62026',
  '-',
  '(우:06595) 서울특별시 서초구 법원로3길 25 (서초동, 태흥빌딩) 2-5층',
  '02-595-4142',
  'leesjlo@naver.com'],
 ['이재용',
  'JY법률사무소',
  '128-32-34769',
  '-',
  '(우:06647) 서울특별시 서초구 서초대로 254 (서초동, 오퓨런스) 14층',
  '025824833',
  'wjdgml1278@naver.com'],
 ['이재용',
  'JY법률사무소',
  '128-32-34769',
  '-',
  '(우:06647) 서울특별시 서초구 서초대로 254 (서초동, 오퓨런스) 14층',
  '025824833',
  'wjdgml1278@naver.com'],
 ['이용희',
  '변호사 이용희 법률사무소',
  '585-70-00218',
  '-',
  '(우:06605) 서울특별시 서초구 서초중앙로 160 (서초동, 법률센터) 505호',
  '02-575-8859',
  'macoshark@naver.com'],
 ['이용희',
  '변호사 이용희 법률사무소',
  '585-70-00218',
  '-',
  '(우:06605) 서울특별시 서초구 서초중앙로 160 (서초동, 법률센터) 505호',
  '02-575-8859',
  'macoshark@naver.com'],
 [' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  '(우:06647) 서울특별시 서초구 반포대로 114 (서초동, 서초S.R Tower)  ',
  '02-583-1126',
  'lawyer_y@naver.com'],
 [

In [95]:
test = pd.DataFrame(all_user)
test

,0,1,2,3,4,5,6,7,8,9
0,김주덕,법무법인 태일,214-86-62026,-,"(우:06595) 서울특별시 서초구 법원로3길 25 (서초동, 태흥빌딩) 2-5층",02-595-4142,leesjlo@naver.com,None,None,None
1,김주덕,법무법인 태일,214-86-62026,-,"(우:06595) 서울특별시 서초구 법원로3길 25 (서초동, 태흥빌딩) 2-5층",02-595-4142,leesjlo@naver.com,None,None,None
2,이재용,JY법률사무소,128-32-34769,-,"(우:06647) 서울특별시 서초구 서초대로 254 (서초동, 오퓨런스) 14층",025824833,wjdgml1278@naver.com,None,None,None
3,이재용,JY법률사무소,128-32-34769,-,"(우:06647) 서울특별시 서초구 서초대로 254 (서초동, 오퓨런스) 14층",025824833,wjdgml1278@naver.com,None,None,None
4,이용희,변호사 이용희 법률사무소,585-70-00218,-,"(우:06605) 서울특별시 서초구 서초중앙로 160 (서초동, 법률센터) 505호",02-575-8859,macoshark@naver.com,None,None,None
5,이용희,변호사 이용희 법률사무소,585-70-00218,-,"(우:06605) 서울특별시 서초구 서초중앙로 160 (서초동, 법률센터) 505호",02-575-8859,macoshark@naver.com,None,None,None
6,,,,,,,,"(우:06647) 서울특별시 서초구 반포대로 114 (서초동, 서초S.R Tower)",02-583-1126,lawyer_y@naver.com
7,,,,,,,,"(우:06647) 서울특별시 서초구 반포대로 114 (서초동, 서초S.R Tower)",02-583-1126,lawyer_y@naver.com
8,,,,,,,,"(우:06620) 서울특별시 서초구 서초대로78길 5 (서초동, 대각빌딩) 11층(...",010-4826-7096,lawyer_jspark@naver.com
9,,,,,,,,"(우:06620) 서울특별시 서초구 서초대로78길 5 (서초동, 대각빌딩) 11층(...",010-4826-7096,lawyer_jspark@naver.com


In [90]:
test.iloc[:][0]

0                                                  김주덕
1                                                  이재용
2                                                  이용희
3    (우:06647) 서울특별시 서초구 반포대로 114 (서초동, 서초S.R Tower)  
4    (우:06620) 서울특별시 서초구 서초대로78길 5 (서초동, 대각빌딩) 11층(...
Name: 0, dtype: object

In [66]:
len(test)

10

In [71]:
test[0]

0    [이재용, JY법률사무소, 128-32-34769, -, (우:06647) 서울특별...
1    [이재용, JY법률사무소, 128-32-34769, -, (우:06647) 서울특별...
2    [이재용, JY법률사무소, 128-32-34769, -, (우:06647) 서울특별...
3    [이재용, JY법률사무소, 128-32-34769, -, (우:06647) 서울특별...
4    [이재용, JY법률사무소, 128-32-34769, -, (우:06647) 서울특별...
5    [이재용, JY법률사무소, 128-32-34769, -, (우:06647) 서울특별...
6    [이재용, JY법률사무소, 128-32-34769, -, (우:06647) 서울특별...
7    [이재용, JY법률사무소, 128-32-34769, -, (우:06647) 서울특별...
8    [이재용, JY법률사무소, 128-32-34769, -, (우:06647) 서울특별...
9    [이재용, JY법률사무소, 128-32-34769, -, (우:06647) 서울특별...
Name: 0, dtype: object

In [402]:
index = ['url',"name","specialty 1","specialty 2","specialty 3","rating","review", "consultations","Title" ,"Company" , "Comapny Regestration","Mail order Business Number","Address","Service center","Mail","Product", "Review",'Consultation 01', "Con01 Rating/Review", "Con01 Discount/Fee", 'Consultation 02', "Con02 Rating/Review", "Con02 Discount/Fee",'Consultation 03', "Con03 Rating/Review", "Con03 Discount/Fee","Review 01 Heading", "Review 01 Rating","Review 01 Text","Review 02 Heading", "Review 02 Rating","Review 02 Text","Review 03 Heading", "Review 03 Rating","Review 03 Text"  ]
len(index)

35

In [403]:
new.index = index

In [404]:
new

,0
url,https://m.kin.naver.com/profile/index.nhn?u=zv...
name,이재용
specialty 1,손해배상
specialty 2,교통사고
specialty 3,형사사건
rating,평균 평점4.9
review,715
consultations,934
Title,이재용
Company,JY법률사무소


In [259]:
test.columns = ['url',"name","specialty 1","specialty 2","specialty 3","rating","review", "consultations","Title" ,"Company" , "Comapny Regestration","Mail order Business Number","Address","Service center","Mail","Product", "Review","Works" ]

In [260]:
test

,url,name,specialty 1,specialty 2,specialty 3,rating,review,consultations,Title,Company,Comapny Regestration,Mail order Business Number,Address,Service center,Mail,Product,Review,Works
0,https://m.kin.naver.com/profile/index.nhn?u=zv...,이재용,손해배상,교통사고,형사사건,평균 평점4.9,715,934,이재용,JY법률사무소,128-32-34769,,"(우:06647) 서울특별시 서초구 서초대로 254 (서초동, 오퓨런스) 14층",025824833,wjdgml1278@naver.com,3,715,"[[신속하고 정확한 형사사건 법률상담, 평균 평점4.9후기391, 판매가10,000..."


In [262]:
test.drop('review',axis = 1)

,url,name,specialty 1,specialty 2,specialty 3,rating,consultations,Title,Company,Comapny Regestration,Mail order Business Number,Address,Service center,Mail,Product,Review,Works
0,https://m.kin.naver.com/profile/index.nhn?u=zv...,이재용,손해배상,교통사고,형사사건,평균 평점4.9,934,이재용,JY법률사무소,128-32-34769,,"(우:06647) 서울특별시 서초구 서초대로 254 (서초동, 오퓨런스) 14층",025824833,wjdgml1278@naver.com,3,715,"[[신속하고 정확한 형사사건 법률상담, 평균 평점4.9후기391, 판매가10,000..."


In [263]:
test

,url,name,specialty 1,specialty 2,specialty 3,rating,review,consultations,Title,Company,Comapny Regestration,Mail order Business Number,Address,Service center,Mail,Product,Review,Works
0,https://m.kin.naver.com/profile/index.nhn?u=zv...,이재용,손해배상,교통사고,형사사건,평균 평점4.9,715,934,이재용,JY법률사무소,128-32-34769,,"(우:06647) 서울특별시 서초구 서초대로 254 (서초동, 오퓨런스) 14층",025824833,wjdgml1278@naver.com,3,715,"[[신속하고 정확한 형사사건 법률상담, 평균 평점4.9후기391, 판매가10,000..."


In [281]:
f = []
works = test["Works"]
for i in works:
    for j in i:
        for m in j:
            f.append(m)
            
f

['신속하고 정확한 형사사건 법률상담',
 '평균 평점4.9후기391',
 '판매가10,000원할인률75%할인 전 가격40,000',
 '성범죄 상담',
 '평균 평점4.9후기232',
 '판매가10,000원할인률75%할인 전 가격40,000',
 '명쾌하고 꼼꼼한 20분 심층 전화상담',
 '평균 평점4.8후기92',
 '판매가20,000원할인률60%할인 전 가격50,000']

In [297]:
ff = pd.Series(f)

ff = ff.T

In [300]:
frames = [test, ff]
result = pd.concat(frames, axis = 1)
result

,url,name,specialty 1,specialty 2,specialty 3,rating,review,consultations,Title,Company,Comapny Regestration,Mail order Business Number,Address,Service center,Mail,Product,Review,Works,0
0,https://m.kin.naver.com/profile/index.nhn?u=zv...,이재용,손해배상,교통사고,형사사건,평균 평점4.9,715,934,이재용,JY법률사무소,128-32-34769,,"(우:06647) 서울특별시 서초구 서초대로 254 (서초동, 오퓨런스) 14층",025824833,wjdgml1278@naver.com,3,715,"[[신속하고 정확한 형사사건 법률상담, 평균 평점4.9후기391, 판매가10,000...",신속하고 정확한 형사사건 법률상담
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,평균 평점4.9후기391
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"판매가10,000원할인률75%할인 전 가격40,000"
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,성범죄 상담
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,평균 평점4.9후기232
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"판매가10,000원할인률75%할인 전 가격40,000"
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,명쾌하고 꼼꼼한 20분 심층 전화상담
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,평균 평점4.8후기92
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"판매가20,000원할인률60%할인 전 가격50,000"


In [162]:
basic

['이재용', '손해배상', '교통사고', '형사사건', '평균 평점4.9', '715', '934']

In [164]:
maininfo

['이재용',
 'JY법률사무소',
 '128-32-34769',
 '',
 '(우:06647) 서울특별시 서초구 서초대로 254 (서초동, 오퓨런스) 14층',
 '025824833',
 'wjdgml1278@naver.com']

In [163]:
all_projects

[['신속하고 정확한 형사사건 법률상담', '평균 평점4.9후기391', '판매가10,000원할인률75%할인 전 가격40,000'],
 ['성범죄 상담', '평균 평점4.9후기232', '판매가10,000원할인률75%할인 전 가격40,000'],
 ['명쾌하고 꼼꼼한 20분 심층 전화상담', '평균 평점4.8후기92', '판매가20,000원할인률60%할인 전 가격50,000']]

In [205]:
products

['3', '715']

# Basic

In [168]:
basic = []

for i in range(1):
#     print("Serial :,"i)
    all_projects = []
    url = pl[1]
    print(url)
    driver = webdriver.Chrome(r'C:\Users\USER\chromedriver_win32\chromedriver.exe')
    driver.get(url)
    time.sleep(3)
    source_code = driver.page_source
    soup = BeautifulSoup(source_code, 'html.parser')
    name = soup.find_all('strong', class_ = 'userName--1ZA07')
    for n in name:
        basic.append(n.text)
        
    category = soup.find_all('strong', class_ = 'introCategory--F81Ky')
    for e in category:
        basic.append(e.text)
        
    sp = soup.find_all('div', class_ = 'categoryName--1zWtA')
    for m in sp:
        basic.append(m.text)
    rating = soup.find_all('div', class_ = 'itemRating--360UA itemRating--2-rFv typeLarge--1cEMN')
    for k in rating:
        basic.append(k.text)
    reviews = soup.find_all('span', class_ = 'statsNum--32OX2')
    for kk in reviews:
        basic.append(kk.text) 
        

    driver.quit()
print(basic)

https://m.kin.naver.com/profile/index.nhn?u=zvRvZSrTBCeYUJjRbEc5OKhzAkyWWga1TYilww4ty7Y%3D
['이재용', '법률', '손해배상', '교통사고', '형사사건', '평균 평점4.9', '715', '934']


In [169]:
basic

['이재용', '법률', '손해배상', '교통사고', '형사사건', '평균 평점4.9', '715', '934']

# Projects

In [186]:
projects = soup.find_all('div', class_ = 'listArea--peDdh')
projects

[<div class="listArea--peDdh"><div class="item--1ZJSx"><a class="itemCard--2Whvq ItemSmallThumbnail_itemCard--196nY productCard--1WjeX" href="/mobile/expert/product/detail?storeId=100002042&amp;productId=100003754"><div class="itemLink--2ljnw"><div class="itemBox--3y657"><div class="itemThumb--Ke4MS"><div class="thumbnail--WbaVj" style='background-image: url("https://phinf.pstatic.net/image.nmv/kin_2020_09_08_5/e2161813-f183-11ea-9635-246e963a4708_01.jpg");'></div></div><div class="itemInfo--24tcX"><div class="itemTitle--2vWBq elip2--nFWXY">신속하고 정확한 형사사건 법률상담</div><div class="itemGroup--2RnIL ItemGroup_itemGroup--1f-on"><div class="itemRating--360UA"><span class="blind">평균 평점</span><svg class="iconStar--2zx5M" height="15" viewbox="0 0 13 12" width="15"><path d="M6.5 9.75l-3.82 2.009.73-4.255L.317 4.491l4.272-.62L6.5 0l1.91 3.87 4.272.621L9.59 7.504l.73 4.255z" fill="#4A65F6" fill-rule="evenodd"></path></svg>4.9</div><div class="itemCount--2HsJv"><em class="countText--3Xuau">후기</em><spa

In [189]:
### Project data for one user
projects = soup.find_all('div', class_ = 'listArea--peDdh')

all_projects = []
for y in projects:
    yy = y.find_all('div', class_ = 'item--1ZJSx')
    for t in yy:
        project_item = []
        tdiv = t.find_all('div', class_ =['itemTitle--2vWBq','elip2--nFWXY'])
        for td in tdiv:
            project_title = td.text
            project_item.append(project_title)
            
        ratdiv = t.find_all('div', class_ =['itemGroup--2RnIL','ItemGroup_itemGroup--1f-on'])
        for rd in ratdiv:
            rating = rd.text
            project_item.append(rating)
            
        feediv = t.find_all('span', class_ ='priceInner--1HE2v')
        for fd in feediv:
            fee = fd.text
            project_item.append(fee)
        
        all_projects.append(project_item)

In [188]:
all_projects

[['신속하고 정확한 형사사건 법률상담', '평균 평점4.9후기391', '판매가10,000원할인률75%할인 전 가격40,000'],
 ['성범죄 상담', '평균 평점4.9후기232', '판매가10,000원할인률75%할인 전 가격40,000'],
 ['명쾌하고 꼼꼼한 20분 심층 전화상담', '평균 평점4.8후기92', '판매가20,000원할인률60%할인 전 가격50,000']]

In [160]:
temp = []

for item in all_projects:
    for i in item:
        hh = ["평균평점","후기","판매가","원할인률","원할인률-", "할인 전 가격"]
        for yt in hh:
            if yt in i:
                temp.append(i.replace(yt ," "))

temp

['평균 평점4.9 391',
 ' 10,000원할인률75%할인 전 가격40,000',
 '판매가10,000원할인률75% 40,000',
 '평균 평점4.9 232',
 ' 10,000원할인률75%할인 전 가격40,000',
 '판매가10,000원할인률75% 40,000',
 '평균 평점4.8 92',
 ' 20,000원할인률60%할인 전 가격50,000',
 '판매가20,000원할인률60% 50,000']

In [113]:
for item in project_item:
    print("명쾌하고" in item)

명쾌하고 꼼꼼한 20분 심층 전화상담
True
평균 평점4.8후기92
False
판매가20,000원할인률60%할인 전 가격50,000
False


# product and review number

In [204]:
products = []

tt = soup.find_all('div', class_ = "list--e6w5E")
for t in tt:  
    cc = t.find_all('div', class_='count--2w5o6')
    for cd in cc:
        cd.find_all('div', class_ = "count--2w5o6")
        ce = cd.text
        print(ce)
        products.append(ce)
products

3
715


['3', '715']

# Seller main information

In [98]:
### Project data for one user
maininfo = []
infos = soup.find_all('ul', class_ = 'productInfoList--1-H-D')
for f in infos:   
    li = f.find_all('li')
    for ll in li:
        uh = ["대표자","상호명","사업자등록번호","통신판매업번호-", "사업장 주소", "고객센터",'메일']
        for u in range(len(uh)):
            if uh[u] in ll.text:
                b = uh[u]
                la = ll.text
                maininfo.append(la.replace(b , ""))   
maininfo

['이재용',
 'JY법률사무소',
 '128-32-34769',
 '',
 '(우:06647) 서울특별시 서초구 서초대로 254 (서초동, 오퓨런스) 14층',
 '025824833',
 'wjdgml1278@naver.com']

# Reviews

In [375]:
### review data for one user
rdiv = soup.find_all('div', class_ = ['titleSection--1lnfz','SubTitle_titleSection--2YQaq'])
for i in rdiv:
    ii = i.find_all('div', class_ = "titleArea--s3-rN")
    for each in ii:
        ai = each.find_all('a', class_ = "titleLink--2BV8L")
        for a in ai:
            print(a)
       

<a class="titleLink--2BV8L" href="/mobile/expert/product/list?storeId=100002042"><span class="linkText--Nm8Me">3개의 상품 더보기</span><svg class="iconCircleArrow--oYTT-" height="18" viewbox="0 0 20 20" width="18"><g fill="none" fill-rule="evenodd"><circle cx="10" cy="10" r="9.5" stroke="#DBDBDB"></circle><path d="M6.5 10h6M11 7l3 3-3 3" stroke="#7E96FF" stroke-linecap="round" stroke-linejoin="round" stroke-width="1.5"></path></g></svg></a>
<a class="titleLink--2BV8L" href="/mobile/expert/product/list?storeId=100002042"><span class="linkText--Nm8Me">더보기</span><svg class="iconCircleArrow--oYTT-" height="18" viewbox="0 0 20 20" width="18"><g fill="none" fill-rule="evenodd"><circle cx="10" cy="10" r="9.5" stroke="#DBDBDB"></circle><path d="M6.5 10h6M11 7l3 3-3 3" stroke="#7E96FF" stroke-linecap="round" stroke-linejoin="round" stroke-width="1.5"></path></g></svg></a>
<a class="titleLink--2BV8L" href="/mobile/expert/review/list?u=zvRvZSrTBCeYUJjRbEc5OKhzAkyWWga1TYilww4ty7Y%3D"><span class="linkTex

In [376]:
rdiv = soup.find_all('div', class_ = "listSection--kViCl")
reviews_rating = []
reviews_heading = []
reviews_text = []

for eachr in rdiv:
    ee = eachr.find_all('div', class_ = "reviewItem--1OwNO")
    for each in ee:
        rating = each.find_all('div', class_ = ["expertPoint--2Zrvr","expertPoint--13H3V"])
        for r in rating:
            reviews_rating.append(r.text)
            
        head = each.find_all('div', class_ = "reviewTitle--qv3Pk")
        for r in head:
            reviews_heading.append(r.text)

        commentdiv = each.find_all('p', class_ = "reviewText--28mzN")
        for ecom in commentdiv:
            reviews_text.append(ecom.text)
            
review_obj = []

for i in range(len(reviews_heading)):
    review_obj.append(reviews_heading[i])
    review_obj.append(reviews_rating[i])
    review_obj.append(reviews_text[i])


In [377]:
reviews_rating

['평점5', '평점5', '평점5']

In [378]:
reviews_heading

['성범죄 상담', '성범죄 상담', '성범죄 상담']

In [379]:
reviews_text

['늦은시간에다 억울함에 말도 더듬게 되고 화가 치밀어오르는데 차분하게 이야기 들어주시고 상담해주셔서 감사합니다 정말 억울한 상황 풀고 싶습니다 감사합니다',
 '늦은시간에 상담감사합니다 ',
 '빠르게 제가 궁금한 부분에 대해서 설명해주십니다!']

In [380]:
review_obj = []

for i in range(len(reviews_heading)):
    review_obj.append(reviews_heading[i])
    review_obj.append(reviews_rating[i])
    review_obj.append(reviews_text[i])

In [381]:
review_obj

['성범죄 상담',
 '평점5',
 '늦은시간에다 억울함에 말도 더듬게 되고 화가 치밀어오르는데 차분하게 이야기 들어주시고 상담해주셔서 감사합니다 정말 억울한 상황 풀고 싶습니다 감사합니다',
 '성범죄 상담',
 '평점5',
 '늦은시간에 상담감사합니다 ',
 '성범죄 상담',
 '평점5',
 '빠르게 제가 궁금한 부분에 대해서 설명해주십니다!']

In [ ]:
# all project dataframe section
# allprojectdf = pd.DataFrame(all_project)
# allprojectdf.columns =["Representative", "Company_name", "Business_registration_number", "Mail_order_number", "Business_address", "Customer_Center",'Mail']
# allprojectdf